##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Versiones C51/Rainbow de DQN

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/9_c51_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/9_c51_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/9_c51_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/9_c51_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

En este ejemplo se muestra cómo usar la biblioteca de TF-Agents para entrenar un agente [Categorical DQN (C51)](https://arxiv.org/pdf/1707.06887.pdf) en el entorno Cartpole.

![Cartpole environment](https://github.com/tensorflow/agents/blob/master/docs/tutorials/images/cartpole.png?raw=1)

Como requisito previo, asegúrese de echar un vistazo al [tutorial DQN](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb). En este tutorial se da por sentado que el usuario está familiarizado con el tutorial de DQN; nos centraremos principalmente en las diferencias entre DQN y C51.


## Preparación


Si todavía no ha instalado tf-agents, ejecute los siguientes comandos:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents
!pip install pyglet

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import categorical_q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## Hiperparámetros

In [ ]:
env_name = "CartPole-v1" # @param {type:"string"}
num_iterations = 15000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_capacity = 100000  # @param {type:"integer"}

fc_layer_params = (100,)

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
gamma = 0.99
log_interval = 200  # @param {type:"integer"}

num_atoms = 51  # @param {type:"integer"}
min_q_value = -20  # @param {type:"integer"}
max_q_value = 20  # @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Entorno

Cargue el entorno como lo hizo antes, con uno para el entrenamiento y otro para la evaluación. Aquí usaremos CartPole-v1 (en lugar de CartPole-v0 como se usa en el tutorial DQN), que tiene una recompensa máxima de 500, mayor que la de 200.

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

C51 es un algoritmo de Q-learning basado en DQN. Al igual que DQN, se puede usar en cualquier entorno que cuente con un espacio de acción discreto.

La principal diferencia entre C51 y DQN es que en lugar de simplemente predecir el valor Q de cada par estado-acción, C51 predice un modelo de histograma para la distribución de probabilidad del valor Q:

![Ejemplo de distribución de C51](images/c51_distribution.png)

Al aprender la distribución en lugar de limitarse al valor esperado, el algoritmo es capaz de mantenerse más estable durante el entrenamiento, lo que se traduce en un mejor rendimiento final. Esto es especialmente cierto en situaciones con distribuciones de valores bimodales o incluso multimodales, en las que una única media no proporciona una imagen precisa.

Para poder entrenar sobre distribuciones de probabilidad en lugar de hacerlo sobre valores, C51 debe hacer algunos cálculos distribucionales complejos para determinar su función de pérdida. Pero no se preocupe, ¡en TF-Agents nos encargamos de todo esto!

Para crear un Agente C51, primero tenemos que crear un `CategoricalQNetwork`. La API de `CategoricalQNetwork` es la misma que la de `QNetwork`, salvo que hay un argumento adicional `num_atoms`. Esto representa el número de puntos de apoyo en nuestras estimaciones de distribución de probabilidad. (La imagen de arriba incluye 10 puntos de apoyo, cada uno representado por una barra azul vertical). Como se puede deducir por el nombre, el número predeterminado de átomos es 51.


In [ ]:
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    num_atoms=num_atoms,
    fc_layer_params=fc_layer_params)

También necesitamos un `optimizer` para entrenar la red que acabamos de crear y una variable `train_step_counter` para hacer un seguimiento de la cantidad de veces que se actualizó la red.

Tenga en cuenta que otra diferencia significativa en comparación con `DqnAgent` vanilla, es que ahora debemos especificar `min_q_value` y `max_q_value` como argumentos. stos especifican los valores más extremos del apoyo (en otras palabras, el más extremo de los 51 átomos a cada lado). Asegúrese de elegirlos correctamente para su entorno específico. Aquí usamos -20 y 20.

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter)
agent.initialize()

Por último, cabe destacar que también incluimos un argumento para utilizar actualizaciones de n pasos con $n$ = 2. En el aprendizaje Q de un solo paso ($n$ = 1), solo se calcula el error entre los valores Q en el paso de tiempo actual y el siguiente paso de tiempo a partir del rendimiento de un solo paso (basado en la ecuación de optimalidad de Bellman). El rendimiento de un solo paso se define de la siguiente manera:

$G_t = R_{t + 1} + \gamma V(s_{t + 1})$

donde se define $V(s) = \max_a{Q(s, a)}$.

Las actualizaciones de n pasos implican la expansión de la función de rendimiento estándar de un solo paso $n$ veces:

$G_t^n = R_{t + 1} + \gamma R_{t + 2} + \gamma^2 R_{t + 3} + \dots + \gamma^n V(s_{t + n})$

Las actualizaciones de n pasos permiten que el agente arranque desde más lejos en el futuro, y con el valor correcto de $n$, esto a menudo se traduce en un aprendizaje más rápido.

Si bien las actualizaciones C51 y de n pasos a menudos se combinan con la repetición priorizada para formar el núcleo del [agente Rainbow](https://arxiv.org/pdf/1710.02298.pdf), no observamos ninguna mejora apreciable al implementar la repetición priorizada. Es más, descubrimos que cuando combinamos nuestro agente C51 con actualizaciones de n pasos, nuestro agente funciona tan bien como otros agentes Rainbow en la muestra de entornos Atari que hemos probado.

## Métricas y evaluación

La métrica más común que se usa para evaluar una política es el rendimiento medio. El rendimiento es la suma de las recompensas obtenidas al ejecutar una política en un entorno durante algunos episodios.  Podemos calcular la métrica de rendimiento medio de la siguiente manera.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

compute_avg_return(eval_env, random_policy, num_eval_episodes)

# Please also see the metrics module for standard implementations of different
# metrics.

## Recopilación de datos

Como se indica en el tutorial de DQN, debe configurar el búfer de repetición y la recopilación inicial de datos con la política aleatoria.

In [ ]:
#@test {"skip": true}
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

def collect_step(environment, policy):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  replay_buffer.add_batch(traj)

for _ in range(initial_collect_steps):
  collect_step(train_env, random_policy)

# This loop is so common in RL, that we provide standard implementations of
# these. For more details see the drivers module.

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)

iterator = iter(dataset)

## Entrenamiento del agente

El bucle de entrenamiento implica tanto la recopilación de datos del entorno como la optimización de las redes del agente. A lo largo del proceso, evaluaremos de vez en cuando la política del agente para ver nuestro rendimiento.

Lo que sigue tardará unos ~7 minutos en ejecutarse.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1:.2f}'.format(step, avg_return))
    returns.append(avg_return)

## Visualización


### Gráficos

Podemos representar gráficamente una comparativa entre el rendimiento y los pasos globales para comprobar el rendimiento de nuestro agente. `Cartpole-v1`, el entorno da una recompensa de +1 por cada paso de tiempo que el poste se mantiene erguido, y como la cantidad máxima de pasos es 500, el máximo rendimiento posible también es 500.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=550)

### Videos

Resulta útil visualizar el funcionamiento de un agente mediante el renderizado del entorno en cada paso. Pero antes de hacerlo, creemos una función para insertar videos en este colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

El siguiente código visualiza la política del agente para algunos episodios:

In [ ]:
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)

C51 tiende a conseguir resultados ligeramente mejores que DQN en CartPole-v1, pero la diferencia entre los dos agentes se hace cada vez más significativa en entornos de creciente complejidad. Por ejemplo, en la prueba Atari 2600, C51 muestra una mejora media del 126% en comparación con DQN después de normalizar con respecto a un agente aleatorio. Se pueden obtener mejoras adicionales si se incluyen actualizaciones de n pasos.

Si desea profundizar en el algoritmo C51, consulte [A Distributional Perspective on Reinforcement Learning (2017)](https://arxiv.org/pdf/1707.06887.pdf).